# Review of Probability

## CSCI E-83
## Stephen Elston

Probability theory is at the core of probabalistic programming. 

## Axioms of probability

All probability distributions must have a certain properties, which we refer to as the **axioms of probability**. These are:

- Probability for any set, A, is bounded between 0 and 1:  

$$0 \le P(A) \le 1 $$
- Probability of the Sample Space = 1:  

$$P(S) = \sum_{All\ i}P(a_i) = 1 $$

- The probability of finite independent unions is the sum of their probabilities:

$$P(A \cup B) = P(A) + P(B)\\ if\ and\ only\ if\\ A \cap B = 0 $$

## Conditional distributions and Bayes' Theorem



## Marginal distributions

We have seen how conditional distributions 

## Independence
